In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

b = chat.predict("How many planets are there?")

b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [ ]:
from langchain.chat_models import ChatOpenAI

# 모델의 창의성 결정: temperature
# low temperature: 창의적이지 않음, 
# high temperature: 창의적, 무작위성을 갖음
chat = ChatOpenAI(
    temperature=0.1
)


In [ ]:
'''
텍스트 predict: 질문을 하고 답변을 받는다.

message 들을 predict 해보자.
우선 message constructor들을 import 한다.

AIMessage: AI에 의해 보내지는 것
SystemMessage: 우리가 LLM에 설정들을 제공하기 위한 Message


predict 와 predict_messages 의 차이?
string을 predict 하는 것과 message 들의 list 인 messages 를 predict 하는 것의 차이.
'''

from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    
    # AI에 일종의 기본 설정, 기본값, 기본 context 를 설정.
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian.",
    ),
    
    # AI 가 이미 메세지를 보낸 것으로 가정
    # - string을 미리 만들어두고 일종의 가상 대화를 만듬.
    AIMessage(
        content="Ciao, mi chiamo Paolo!",
    ),
    
    # 사람이 질문한다고 가정
    # - 사용자로서 질문
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, What is your name?",
    ),
]

# 단순히 string들을 predict 하는 것 대신에 
# predict_messages 를 사용하고 message의 list를 넘겨준다.
chat.predict_messages(messages)



AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

In [ ]:
""" python
Prompt Templates

# 설정값들을 전달하고 싶지 않은 경우
PromptTemplate: 그냥 string을 이용해서 template 을 만든다.
ex) 
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}.",
)

먼저 template 를 만들고, format메서드의 호출 결과를 prompt에 저장.

"""

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 설정값들을 전달하고 싶지 않은 경우
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}.",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)
'''
위 과정에서 template.format 과 predict를 반복해서 작성하는게 귀찮다고 느낄 수 있다.
뒤에 template을 format한 후 chat model에게 전달하고 predict를 호출하는 과정까지 
말 그대로 한 줄의 코드로 구현하게 될 것이다.
이걸 먼저 하지 않는 이유는 이걸 보면 말 그대로 마법을 보는 것 같기 떄문에,
각 component(구성요소) 를 이해하기 어렵기 때문이야
 '''

'What is the distance between Mexico and Thailand.'

In [3]:
""" python
Prompt Templates

# 설정값 전달
ChatPromptTemplate: template을 message로부터 만든다.
ex) 

먼저 template 를 만들고, format메서드의 호출 결과를 prompt에 저장.

"""

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 설정값 전달
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}.",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [6]:
"""
# ChatPromptTemplate.from_messages
똑같이 messages list를 전달할 거지만, 좀 더 변형을 한다.

('message type', 'content')

시스템메세지 : GPT에게 보내는 프롬프트
AI메세지     :  
휴먼메세지   :  

SystemMessage(content="You are a geography expert. And you only reply in Italian."),
-> ("system", "You are a geography expert. And you only reply in Italian.")

AIMessage(content="Ciao, mi chiamo Paolo!") 
-> ("ai", "Ciao, mi chiamo Paolo!")
    
HumanMessage(content="What is the distance between Mexico and Thailand. Also, What is your name?)
-> ("human", "What is the distance between Mexico and Thailand. Also, What is your name?")

"""

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    (
        "human",
        "What is the distance between {country_a} and {country_b}. Also, What is your name?. Also, what is your name?",
    ),
])

prompt = template.format_messages(
    language="korean",
    name="Socrates",
    country_a="Mexico",
    country_b="Thiland",
    )

chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 멕시코와 태국 사이의 거리는 약 16,000km입니다. 제 이름은 소크라테스입니다.')

In [ ]:
""" OutputParser 와 LangChain expression language(표현언어)
LangChain expression language(표현언어)는 우리의 코드를 굉장히 줄여줄 것이다.
그리고 다양한 template 와 llm 호출, 그리고 서로 다른 응답(response)을 함께 사용하게 해준다.

1. Output Parser 만들기
Output Parser가 필요한 이유?
- llm의 응답을 변형해야 할 때가 있기 때문, 
- 이번에는 응답을 list 로 변환시키는 것을 만들어 보자.


"""

from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)
'''output parse
LLM 으로부터 성공적으로 응답 받음
- prompt template 사용.
- prompt template 포맷팅 한 후 output Parser 사용.

'''

content='red, blue, green, yellow, orange, purple, pink, black, white, brown'


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [ ]:
"""더 나은 방법
우리에게 필요한 건 template 와 output parser 그리고 chat model

- 체인 chain 만들기
'chain'이라는 것은 기본적으로, 모든 요소를 합쳐주는 역할을 한다.
합쳐진 요소들은 하나의 체인으로 실행된다. 하나하나 순서대로 result를 반환할때까지.

- 랭체인의 | 기호 사용.
chain = template | chat | CommaOutputParser()
내부에서 랭체인이 우릴 위해 .format_messages를 호출하고, 그 다음에 chat.predict도 호출하고,
마지막으로 parse를 호출한다. 
이 방법으로 훨씬 더 멋진 일들을 많이 할 수 있다.
왜냐하면 이걸로 체인끼리 결합할 수 있기 때문.

- LangChain expression language 을 통한 결합
chain_one = template | chat | CommaOutputParser()
chain_two = template_2 | chat | outputparser
all = chain_one | chain_two | output
체인 원의 출력값을 체인 투의 입력값으로 사용 가능, 그리고 또 다른 output parse 사용 가능.

이런식으로

"""

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

chain = template | chat | CommaOutputParser()
# chain 동작
chain.invoke(
    {
        "max_items": 5,
        "question": "what are the pokemons?",
    }
)

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

# LangChain expression language
## 두 개의 체인 생성
- 체인끼리 연결
- 하나의 체인 실행, 그 출력값을 이용해 다른 체인을 실행할 수 있다(순차적).

## 쉐프 template
1. 레시피를 전달해주는 셰프
- 랭체인의 prompt 중 chatprompttemplate 를 import 해와서 template 생성
-   
2. 전달받은 레시피를 채식주의자 레시피로 바꿔주는 셰프


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuising with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

{recipe}가 하나의 입력값이 된다.  
그 다음은 {cuisine}.

chef_chain이 첫 번째 chain 이 된다.  
왜냐하면 먼저 레시피(recipe) 값을 얻고, 그 다음 채식주의 레시피로 변경할 거니까.  
chef_chain 다음에 veg_chain 이 와야된다는게 자연스럽게 이해될 것이다.  

** 한 가지 더 생각할 점은 입려값과 출력값의 이름이 일치해야 한다는 것!! **


In [27]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine":"indian",
})

Great choice! Here's a simple recipe for Chicken Tikka Masala, a popular Indian dish that is sure to impress your friends and family.

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, and chili powder. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. In a large skillet, heat the vegetable oil ov

AIMessageChunk(content="To make this Chicken Tikka Masala recipe vegetarian, we can replace the chicken with a plant-based alternative such as tofu or seitan. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or seitan, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground turmeric\n- 1 teaspoon garam masala\n- 1 teaspoon ground coriander\n- 1 teaspoon chili powder (adjust to taste)\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup coconut cream (or another dairy-free heavy cream alternative)\n- Salt and pepper to taste\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Instead of marinating chicken, marinate the tofu or seitan in the yogurt and spice mixture as instructed in the recipe. Refrigerate

# 두 개의 체인
그 중 하나는 chef_chain
chef_prompt, chat, 그리고 LLM 으로 이루어져있다.

그리고 다른 하나인 veg_chain 도 역시 beg_chef_prompt 와 chat language model 을 가지고 있다.

여기서 우리가 하려던 것은 이 둘을 묶은 chain 을 만드는 것이다.  
chef_chain의 출력값이 veg_chain의 입력값이 되게끔 ...
그래서 final_chain 을 만들었다.


